# 결정 트리 실습

사용자 행동 인식 데이터 세트

[원본 데이터셋]
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

데이터 세트 정보:

실험은 19-48세 사이의 30명의 지원자 그룹으로 수행되었습니다. 각자 스마트폰(Samsung Galaxy S II)을 허리에 차고 6가지 활동(
1:WALKING, 
2:WALKING_UPSTAIRS, 
3:WALKING_DOWNSTAIRS, 4:SITTING, 5:STANDING, 6:LAYING)을 수행했습니다. 내장된 가속도계와 자이로스코프를 사용하여 50Hz의 일정한 속도로 3축 선형 가속도와 3축 각속도를 캡처했습니다. 실험은 데이터에 수동으로 레이블을 지정하기 위해 비디오로 녹화되었습니다. 얻은 데이터 세트는 훈련 데이터 생성을 위해 70%, 테스트 데이터 생성을 위해 30%가 선택된 두 세트로 무작위로 분할되었습니다.

센서 신호(가속도계 및 자이로스코프)는 노이즈 필터를 적용하여 사전 처리된 다음 2.56초 및 50% 중첩(128 판독/창)의 고정 너비 슬라이딩 창에서 샘플링되었습니다. 중력 및 신체 운동 성분을 갖는 센서 가속도 신호는 Butterworth 저역 통과 필터를 사용하여 신체 가속도와 중력으로 분리되었습니다. 중력은 저주파 성분만 있다고 가정하므로 차단 주파수가 0.3Hz인 필터를 사용했습니다. 각 창에서 시간 및 주파수 영역에서 변수를 계산하여 특징 벡터를 얻었습니다.

이 데이터 세트에 대한 자세한 내용은 README.txt 파일을 확인하십시오.

참가자 중 한 명과 함께 녹화된 6가지 활동의 예를 포함하는 실험 비디오는 다음 링크에서 볼 수 있습니다.

https://www.youtube.com/watch?v=XOEN9W05_4A]


**Mission**

수집된 데이터를 기반으로 결정트리를 이용해 어떤 행동을 하고 있는지 예측해 봅시다.

In [ ]:
# 녹화된 6가지 활동: 
# 앉아있기, 일어나기, 눕기, 복도를 일자로 걷기, 계단 내려가기, 계단 올라가기

In [ ]:
# ## 코랩을 사용할 때
# #드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# #현재 작업 위치 이동
# #띄어쓰기에 \붙일 것
# %cd /content/drive/Othercomputers/내\ 노트북_before/Devpy/13.머신러닝

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 데이터 로드

1. 피처만 있는 파일을 로드
2. 훈련데이터와 피처를 합쳐 훈련 데이터 프레임 만듬
3. 데스트데이터와 피처를 합쳐 테스트 데이터 프레임 만듬

In [2]:
# 피처이름 로드 
feature_name_df = pd.read_csv('./dataset/human_activity/features_new.txt', header=None,names=['column_name'])
feature_name_df 

,column_name
0,tBodyAcc-mean()-X
1,tBodyAcc-mean()-Y
2,tBodyAcc-mean()-Z
3,tBodyAcc-std()-X
4,tBodyAcc-std()-Y
...,...
556,"angle(tBodyGyroMean,gravityMean)"
557,"angle(tBodyGyroJerkMean,gravityMean)"
558,"angle(X,gravityMean)"
559,"angle(Y,gravityMean)"


In [3]:
# 피처이름을 리스트로
feature_name = feature_name_df['column_name'].values.tolist()
feature_name

['tBodyAcc-mean()-X',
 'tBodyAcc-mean()-Y',
 'tBodyAcc-mean()-Z',
 'tBodyAcc-std()-X',
 'tBodyAcc-std()-Y',
 'tBodyAcc-std()-Z',
 'tBodyAcc-mad()-X',
 'tBodyAcc-mad()-Y',
 'tBodyAcc-mad()-Z',
 'tBodyAcc-max()-X',
 'tBodyAcc-max()-Y',
 'tBodyAcc-max()-Z',
 'tBodyAcc-min()-X',
 'tBodyAcc-min()-Y',
 'tBodyAcc-min()-Z',
 'tBodyAcc-sma()',
 'tBodyAcc-energy()-X',
 'tBodyAcc-energy()-Y',
 'tBodyAcc-energy()-Z',
 'tBodyAcc-iqr()-X',
 'tBodyAcc-iqr()-Y',
 'tBodyAcc-iqr()-Z',
 'tBodyAcc-entropy()-X',
 'tBodyAcc-entropy()-Y',
 'tBodyAcc-entropy()-Z',
 'tBodyAcc-arCoeff()-X,1',
 'tBodyAcc-arCoeff()-X,2',
 'tBodyAcc-arCoeff()-X,3',
 'tBodyAcc-arCoeff()-X,4',
 'tBodyAcc-arCoeff()-Y,1',
 'tBodyAcc-arCoeff()-Y,2',
 'tBodyAcc-arCoeff()-Y,3',
 'tBodyAcc-arCoeff()-Y,4',
 'tBodyAcc-arCoeff()-Z,1',
 'tBodyAcc-arCoeff()-Z,2',
 'tBodyAcc-arCoeff()-Z,3',
 'tBodyAcc-arCoeff()-Z,4',
 'tBodyAcc-correlation()-X,Y',
 'tBodyAcc-correlation()-X,Z',
 'tBodyAcc-correlation()-Y,Z',
 'tGravityAcc-mean()-X',
 'tGravityA

### 데이터셋

In [4]:
# 학습(X_train) 피처 데이터 셋과 테스트(X_test) 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
X_train = pd.read_csv('./dataset/human_activity/train/X_train.txt',sep='\s+', names=feature_name )
X_test = pd.read_csv('./dataset/human_activity/test/X_test.txt',sep='\s+', names=feature_name)

In [10]:
X_train.shape # 561개의 피처와 7352개의 샘플

(7352, 561)

In [11]:
X_test.shape # 561개의 피처와 2947개의 샘플

(2947, 561)

In [5]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
y_train = pd.read_csv('./dataset/human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
y_test = pd.read_csv('./dataset/human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])

In [13]:
y_train.shape

(7352, 1)

In [14]:
y_test.shape

(2947, 1)

In [15]:
#action 피처의 class별 갯수
y_train['action'].value_counts()

6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: action, dtype: int64

In [16]:
y_test['action'].value_counts()

6    537
5    532
1    496
4    491
2    471
3    420
Name: action, dtype: int64

In [ ]:
# 이 때 [action]컬럼의 두 데이터의 경우를 모두 합쳐 카운트해야하나요? 

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# DesisionTree분류 클래스 사용(randomstate 156)
dt_clf = DecisionTreeClassifier(random_state=156)

# 훈련
dt_clf.fit(X_train , y_train)


# 예측
pred = dt_clf.predict(X_test)


#평가
'예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)) 

'예측 정확도: 0.8548'

In [19]:
# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


## 하이퍼파라미터 튜닝1

In [7]:
from sklearn.model_selection import GridSearchCV

#max_depth를 바꿔가며 훈련해보자
#나무의 깊이를 얼마만큼 성장시킬것인가

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}


In [8]:
# GridSearchCV 사용
import pandas as pd
grid_dt_clf = GridSearchCV(dt_clf, param_grid=params, cv=3, refit=True) 


# 훈련
grid_dt_clf.fit(X_train, y_train)

# 38.5s 소요

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20, 24]})

In [9]:
# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성. 
scores_df = pd.DataFrame(grid_dt_clf.cv_results_) 
scores_df # rank = 1 : 최적의 파라미터

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.210377,0.011808,0.006835,0.000850,6,{'max_depth': 6},0.839249,0.868217,0.834286,0.847251,0.014963,1
1,1.443584,0.081654,0.006334,0.000236,8,{'max_depth': 8},0.829457,0.843329,0.839184,0.837323,0.005814,2
2,1.768474,0.139047,0.011504,0.007431,10,{'max_depth': 10},0.831905,0.850265,0.808163,0.830111,0.017235,3
3,2.299233,0.430094,0.007168,0.000624,12,{'max_depth': 12},0.838433,0.835985,0.808571,0.827663,0.013537,6
4,2.042188,0.146411,0.006668,0.000624,16,{'max_depth': 16},0.822521,0.838841,0.814694,0.825352,0.010059,7
5,2.036354,0.153352,0.006501,0.000408,20,{'max_depth': 20},0.833129,0.838841,0.813061,0.828344,0.011055,4
6,2.058357,0.155321,0.006668,0.000624,24,{'max_depth': 24},0.833129,0.838841,0.813061,0.828344,0.011055,4


In [10]:
print('GridSearchCV 최적 파라미터:', grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 6}
GridSearchCV 최고 정확도: 0.8473


In [11]:
#베스트 분류기 선택
estimator = grid_dt_clf.best_estimator_
#예측
pred = estimator.predict(X_test)

#평가
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred))) 

테스트 데이터 세트 정확도: 0.8558


```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?

    : 정확도가 85.48%에서 85.58% 로 약간 향상되었습니다.
```

## 하이퍼파라미터 튜닝2

In [12]:
#하이퍼파라미터
params = {
    'max_depth' : [ 8 , 12, 16 ,20], 
    'min_samples_split' : [16,24],
}

In [13]:
# GridSearchCV 사용
import pandas as pd
grid_dt_clf = GridSearchCV(dt_clf, param_grid=params, cv=3, refit=True) 


# 훈련
grid_dt_clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [8, 12, 16, 20],
                         'min_samples_split': [16, 24]})

In [14]:
print('GridSearchCV 최적 파라미터:', grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 8, 'min_samples_split': 16}
GridSearchCV 최고 정확도: 0.8459


In [15]:
#베스트 분류기 선택
estimator = grid_dt_clf.best_estimator_
#예측
pred = estimator.predict(X_test)

#평가
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred))) 

테스트 데이터 세트 정확도: 0.8717


```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?

    :85.48%-> 85.58% -> 87.17% 로 알고리즘 성능이 개선되었습니다.
```

---

**[생활탐구]**


```
방문을 꼭 닫은채 공부하고 있다는 보검이...
정말 보검이는 공부하고 있을까요?
생체 신호 측정기를 착용한 보검이로 부터 아래와 같은 신호를 가져왔습니다. 
지금 현재 보검이는 무엇을 하고 있나요?

[보검이 생체 신호 파일]X_quiz.txt 

```

In [18]:
#신호 읽어서 예측해보기
X_quiz55 = pd.read_csv('./dataset/human_activity/test/X_quiz55.txt', names= feature_name)
X_quiz55

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.173769,-0.024869,-0.094733,-0.893862,-0.944204,-0.960824,-0.904192,-0.950321,-0.967834,-0.923737,...,-0.267475,0.199002,-0.324917,-0.089103,0.10531,0.183368,-0.169434,0.751465,-0.484928,-0.505


In [19]:
# 예측
pred = estimator.predict(X_quiz55)
pred

array([6], dtype=int64)

In [27]:
y_train['action'].unique() # 0번 값 없음

array([5, 4, 6, 1, 3, 2], dtype=int64)

>* 결과 : 보검이의 생체신호는 6번(계단 올라가기)입니다.
>* 방에서 보검이는 자리에 앉아 공부하지않고 올라가는 동작의 신체활동을 할 것으로 예측됩니다.

In [23]:
#신호 읽어서 예측해보기
X_quiz = pd.read_csv('./dataset/human_activity/test/X_quiz.txt', names= feature_name)

# 예측
pred = estimator.predict(X_quiz)
pred

array([3, 5], dtype=int64)

>* 결과 : 보검이의 생체신호는 3번(눕기)와 5번(계단 내려가기)으로 예측됩니다.
>* 방에서 보검이는 자리에 앉아 공부하지않고, 침대에 누워있다가 내려온 것으로 예측됩니다.

# 실패코드 정리

In [28]:
X_test_df1 = pd.read_csv('./dataset/human_activity/test/X_test.txt',header=None,names=feature_name) # 테스트용 #2947 rows × 561 columns
X_test_df1
# 첫번째 컬럼에 모든 데이터가 들어감 -> 띄어쓰기를 기준으로 split 응용하려고 접근

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,2.5717778e-001 -2.3285230e-002 -1.4653762e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.8602671e-001 -1.3163359e-002 -1.1908252e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.7548482e-001 -2.6050420e-002 -1.1815167e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.7029822e-001 -3.2613869e-002 -1.1752018e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.7483295e-001 -2.7847788e-002 -1.2952716e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,3.1015462e-001 -5.3391250e-002 -9.9108716e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2943,3.6338465e-001 -3.9214016e-002 -1.0591509e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2944,3.4996609e-001 3.0077442e-002 -1.1578796e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2945,2.3759383e-001 1.8466870e-002 -9.6498932e-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 학습(X_train) 피처 데이터 셋과 테스트(X_test) 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용

In [29]:
import numpy as np

X_train_df1 = pd.read_csv('./dataset/human_activity/train/X_train.txt',header=None,names=feature_name) 

X_train_data_list =[]
for i in range(7352) :
    x = X_train_df1['tBodyAcc-mean()-X'][i].split()
    X_train_data_list.append(x)

X_train_data =  np.array(X_train_data_list)

X_train_df = pd.DataFrame(data=X_train_data, columns=feature_name)

In [31]:
X_test_df1 = pd.read_csv('./dataset/human_activity/test/X_test.txt',header=None,names=feature_name) 

X_test_data_list =[]
for i in range(561) :
    x = X_test_df1['tBodyAcc-mean()-X'][i].split()
    X_test_data_list.append(x)

X_test_data =  np.array(X_test_data_list)

X_test_df = pd.DataFrame(data=X_test_data, columns=feature_name)

In [32]:
X_train_df.head() # 데이터프레임에 데이터가 재배치됨

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,2.8858451e-001,-2.0294171e-002,-1.3290514e-001,-9.9527860e-001,-9.8311061e-001,-9.1352645e-001,-9.9511208e-001,-9.8318457e-001,-9.2352702e-001,-9.3472378e-001,...,-7.4323027e-002,-2.9867637e-001,-7.1030407e-001,-1.1275434e-001,3.0400372e-002,-4.6476139e-001,-1.8445884e-002,-8.4124676e-001,1.7994061e-001,-5.8626924e-002
1,2.7841883e-001,-1.6410568e-002,-1.2352019e-001,-9.9824528e-001,-9.7530022e-001,-9.6032199e-001,-9.9880719e-001,-9.7491437e-001,-9.5768622e-001,-9.4306751e-001,...,1.5807454e-001,-5.9505094e-001,-8.6149931e-001,5.3476955e-002,-7.4345661e-003,-7.3262621e-001,7.0351059e-001,-8.4478760e-001,1.8028889e-001,-5.4316717e-002
2,2.7965306e-001,-1.9467156e-002,-1.1346169e-001,-9.9537956e-001,-9.6718701e-001,-9.7894396e-001,-9.9651994e-001,-9.6366837e-001,-9.7746859e-001,-9.3869155e-001,...,4.1450281e-001,-3.9074815e-001,-7.6010372e-001,-1.1855926e-001,1.7789948e-001,1.0069921e-001,8.0852908e-001,-8.4893347e-001,1.8063731e-001,-4.9117815e-002
3,2.7917394e-001,-2.6200646e-002,-1.2328257e-001,-9.9609149e-001,-9.8340270e-001,-9.9067510e-001,-9.9709947e-001,-9.8274984e-001,-9.8930250e-001,-9.3869155e-001,...,4.0457253e-001,-1.1729020e-001,-4.8284451e-001,-3.6787973e-002,-1.2892494e-002,6.4001104e-001,-4.8536645e-001,-8.4864938e-001,1.8193476e-001,-4.7663183e-002
4,2.7662877e-001,-1.6569655e-002,-1.1536185e-001,-9.9813862e-001,-9.8081727e-001,-9.9048163e-001,-9.9832113e-001,-9.7967187e-001,-9.9044113e-001,-9.4246912e-001,...,8.7753013e-002,-3.5147093e-001,-6.9920515e-001,1.2332005e-001,1.2254196e-001,6.9357829e-001,-6.1597061e-001,-8.4786525e-001,1.8515116e-001,-4.3892254e-002


In [ ]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여

In [33]:
y_train_df = pd.read_csv('./dataset/human_activity/train/y_train.txt', header=None,names=['action'])
y_test_df = pd.read_csv('./dataset/human_activity/test/y_test.txt', header=None,names=['action'])

In [34]:
type(X_train_data), type(X_train_df), 

(numpy.ndarray,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [36]:
type(y_train_df), type(y_test_df)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [38]:
X_test_df.shape

(561, 561)

In [39]:
y_test_df.shape

(2947, 1)

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# DesisionTree분류 클래스 사용(randomstate 156)
dt_clf = DecisionTreeClassifier(random_state=156)

# 훈련
dt_clf.fit(X_train_df , y_train_df)


# 예측
pred = dt_clf.predict(X_test_df)


#평가
'예측 정확도: {0:.4f}'.format(accuracy_score(y_test_df, pred)) # y_test_df 에서 ValueError : shape 확해보니 샘플 수가 안맞아서 에러

ValueError: Found input variables with inconsistent numbers of samples: [2947, 561]